In [18]:
from game import Game
import time
import os
import re

## Load maps

In [19]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [20]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [21]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [22]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [23]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [24]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [25]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [26]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

### BFS

In [27]:
# TODO: Must return moves (if there is no solution return None), number of visited states
from collections import deque

def solver_bfs(game_map):
    
    game = Game(game_map)
    initial_state  = {
        'mike_position': game.get_player_position(),
        'box_positions': game.get_box_locations(),
    }
    
    queue = deque([(initial_state, '')])
    visited_state = set()
    visited_count = 0
    moves = ['U', 'D', 'R', 'L']

    while queue:
        state, path = queue.popleft()
        
        visited_count += 1
        
        state_tuple = (state['mike_position'], tuple(state['box_positions']))
        if state_tuple in visited_state:
            continue
        
        if game.is_game_won():
            return path, visited_count
        

        visited_state.add(state_tuple)

        for move in moves:
            if game.apply_move(move):
                new_state = {
                    'mike_position': game.get_player_position(),
                    'box_positions': game.get_box_locations(),
                }
                new_path = path + move
                queue.append((new_state, new_path))
            game.set_box_positions(state['box_positions'])
            game.set_player_position(state['mike_position'])

    return None, 0

### DFS

In [28]:
def solver_dfs(game_map):
    
    game = Game(game_map)
    initial_state = {
        'mike_position': game.get_player_position(),
        'box_positions': game.get_box_locations(),
    }
    
    visited_state = set()
    visited_count = 0
    moves = ['U', 'R', 'D', 'L']

    def dfs(state, path):
        nonlocal visited_count

        if game.is_game_won():
            return path
        
        state_tuple = (state['mike_position'], tuple(state['box_positions']))
        if state_tuple in visited_state:
            return None
        
        visited_state.add(state_tuple)
        visited_count += 1
        for move in moves:
            if game.apply_move(move):
                new_state = {
                    'mike_position': game.get_player_position(),
                    'box_positions': game.get_box_locations(),
                }
                result = dfs(new_state, path + move)
                if result:
                    return result
            game.set_box_positions(state['box_positions'])
            game.set_player_position(state['mike_position'])
                
        return None

    founded_path = dfs(initial_state,'')
    if founded_path:
        return founded_path, visited_count

    return None, 0


### IDS

In [29]:
# TODO: Must return moves, number of visited states

def solver_ids(game_map):
    game = Game(game_map)
    initial_state = {
        'mike_position': game.get_player_position(),
        'box_positions': game.get_box_locations(),
    }
    length, width = game.get_map_size()
    max_depth = length * width
        
    visited_count = 0
    moves = ['U', 'R', 'D', 'L']

    def dls(state, path, depth_limit, visited_state):
        nonlocal visited_count

        if game.is_game_won():
            return path
        
        if depth_limit == 0:
            return None
        
        state_tuple = (state['mike_position'], tuple(state['box_positions']))
        if state_tuple in visited_state:
            return None
        
        visited_state.add(state_tuple)
        visited_count += 1

        for move in moves:
            if game.apply_move(move):
                new_state = {
                    'mike_position': game.get_player_position(),
                    'box_positions': game.get_box_locations(),
                }
                s = visited_state
                result = dls(new_state, path + move, depth_limit-1, visited_state)
                if result:
                    return result
            game.set_player_position(state['mike_position'])
            game.set_box_positions(state['box_positions'])
        
        visited_state.remove(state_tuple)

    
    for i in range(max_depth):
        visited_state = set()
        visited_count = 0
        result = dls(initial_state, '', i, visited_state)
        if result:
            return result, visited_count
        
    return None, 0

At first, the Manhattan distance heuristic was considered. This heuristic, due to the fact that movements only occur in four directions on the game map, never results in a distance greater than the actual distance. From this perspective, this method is **admissible**.  
This method is also **consistent** because every move in the horizontal or vertical direction reduces the movement cost and the distance to the goal by the same or a smaller amount than the previous value.

One of the issues with the Manhattan distance is the consideration of walls, and this problem becomes more significant in maps that have portals because a box might be trapped in an area, and the only way to get the box out of it is through a portal.

To solve this mentioned problem, we define another heuristic.  
In this heuristic, the distance from the box to the portal, from the portal to the goal, and the distance from Mike to the portal are all considered.  
This method is also **consistent** and **admissible** for the reasons mentioned.


### A*

In [30]:
import heapq


def heuristic(game):
    goal_positions = game.get_goal_locations()
    box_positions = game.get_box_locations()
    mike_position = game.get_player_position()
    portal_groups = game.get_portal_locations() 

    total_distance = 0

    for box, goal in zip(box_positions, goal_positions):
        min_distance = abs(box[0] - goal[0]) + abs(box[1] - goal[1])
        box_to_ports = []
        mike_to_ports = []
        for portal_group in portal_groups:
            entry, exit = portal_group
            box_entry_dist_1 = abs(box[0] - entry[0]) + abs(box[1] - entry[1])
            goal_exit_dist_1 = abs(goal[0] - exit[0]) + abs(goal[1] - exit[1])
            goal_exit_dist_2 = abs(box[0] - exit[0]) + abs(box[1] - exit[1])
            goal_entry_dist_2 = abs(goal[0] - entry[0]) + abs(goal[1] - entry[1])
            mike_entry_dist = abs(mike_position[0] - entry[0]) + abs(mike_position[1] - entry[1])
            mike_exit_dist = abs(mike_position[0] - exit[0]) + abs(mike_position[1] - exit[1])
            box_to_ports.append(min(box_entry_dist_1+goal_exit_dist_1, goal_exit_dist_2+goal_entry_dist_2))
            mike_to_ports.append(min(mike_entry_dist, mike_exit_dist))

        if(len(portal_groups) == 2):
            total_distance += min(box_to_ports[0], box_to_ports[1]) + min(mike_to_ports[0], mike_to_ports[1])
        elif(len(portal_groups) == 1):
            total_distance += min(min_distance, box_to_ports[0] + mike_to_ports[0])
        else:
            total_distance += min_distance

    mike_to_box_distance = min(abs(mike_position[0] - box[0]) + abs(mike_position[1] - box[1]) for box in box_positions)

    return total_distance + mike_to_box_distance


def solver_astar(game_map, heuristic_func=heuristic, weight=1):
    game = Game(game_map)

    initial_state = (
        game.get_player_position(),
        tuple(game.get_box_locations())
    )

    priority_queue = []
    initial_h = weight * heuristic_func(game)
    heapq.heappush(priority_queue, (initial_h, 0, initial_state, ''))  

    visited_states = {}
    visited_count = 0
    moves = ['U', 'R', 'D', 'L']

    while priority_queue:
        _, g, state_tuple, path = heapq.heappop(priority_queue)
        visited_count += 1

        mike_position, box_positions = state_tuple

        if state_tuple in visited_states and visited_states[state_tuple] <= g:
            continue
        visited_states[state_tuple] = g

        if set(box_positions) == set(game.get_goal_locations()):
            return path, visited_count  

        for move in moves:
            game.set_player_position(mike_position)
            game.set_box_positions(list(box_positions))
            if game.apply_move(move):  
                new_state_tuple = (
                    game.get_player_position(),
                    tuple(game.get_box_locations())
                )
                _h = heuristic_func(game)
                _g = g + 1  
                _f = (weight * _h) + _g
                heapq.heappush(priority_queue, (_f, _g, new_state_tuple, path + move))
    
    return None, visited_count


## Solve

In [31]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar,
    "Weighted-A*": solver_astar,
}

In [32]:
def solve(map, method, weight=1):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    if method == "Weighted-A*":
        moves, numof_visited_states = SOLVERS[method](game_map, weight = weight)
    else:
        moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [33]:
def solve_all():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        for method in SOLVERS.keys():
            if((method == "A*")):
                solve(map, method)
            

**📝 The results have been recorded in the table of the <span style="color:Red;">report.ipynb</span> file.**


In [ ]:
# for map in [1, 2, 3, 4, 5, 6, 7, 8, 10]:
#     solve(map, "BFS")

# for map in [1, 2, 3, 4, 5]:
#     solve(map, "DFS")

# for map in [1, 2, 3, 4, 5, 8]:
#     solve(map, "IDS")

for map in [1, 2, 3, 4, 5, 6, 7, 8, 10]:
    solve(map, "A*")

for map in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    solve(map, "Weighted-A*", weight=2.5)
    

A* took 0.0 seconds on map 1 and visited 74 states.
7 moves were used: DULRRLU
A* took 0.0 seconds on map 2 and visited 24 states.
6 moves were used: LDULRU
A* took 0.0 seconds on map 3 and visited 101 states.
14 moves were used: ULUURDDDDUULDD
A* took 0.0 seconds on map 4 and visited 3 states.
No Solution Found!
A* took 0.01 seconds on map 5 and visited 1029 states.
15 moves were used: LULDDRDLLUUURUL


A* took 0.17 seconds on map 6 and visited 13823 states.
34 moves were used: DDDDDRRRLLLLLLLRUUUUUUUUUULRRRRRRR
A* took 1.64 seconds on map 7 and visited 85579 states.
34 moves were used: RURRDDDDLDRUUUULLLRDRDRDDLLDLLURLU
A* took 0.14 seconds on map 8 and visited 6025 states.
14 moves were used: UURDLDRRDRURDR
A* took 1.6 seconds on map 10 and visited 99453 states.
29 moves were used: RRRRURDRDLLLRRRUULLLDRRUULULD
Weighted-A* took 0.0 seconds on map 1 and visited 14 states.
7 moves were used: UDLRRLD
Weighted-A* took 0.0 seconds on map 2 and visited 10 states.
6 moves were used: LUDLRD
Weighted-A* took 0.0 seconds on map 3 and visited 44 states.
16 moves were used: ULDUUURDDDDUULDD
Weighted-A* took 0.0 seconds on map 4 and visited 3 states.
No Solution Found!
Weighted-A* took 0.0 seconds on map 5 and visited 91 states.
15 moves were used: LULDLRDRDLLULUU
Weighted-A* took 0.05 seconds on map 6 and visited 5312 states.
34 moves were used: RRDDDDDRLLLLLLLUUUUUUUUURULRRRRRRR
Weighted-A* to